# Project Euler: Problem 59

https://projecteuler.net/problem=59

Each character on a computer is assigned a unique code and the preferred standard is ASCII (American Standard Code for Information Interchange). For example, uppercase A = 65, asterisk (*) = 42, and lowercase k = 107.

A modern encryption method is to take a text file, convert the bytes to ASCII, then XOR each byte with a given value, taken from a secret key. The advantage with the XOR function is that using the same encryption key on the cipher text, restores the plain text; for example, 65 XOR 42 = 107, then 107 XOR 42 = 65.

For unbreakable encryption, the key is the same length as the plain text message, and the key is made up of random bytes. The user would keep the encrypted message and the encryption key in different locations, and without both "halves", it is impossible to decrypt the message.

Unfortunately, this method is impractical for most users, so the modified method is to use a password as a key. If the password is shorter than the message, which is likely, the key is repeated cyclically throughout the message. The balance for this method is using a sufficiently long password key for security, but short enough to be memorable.

Your task has been made easy, as the encryption key consists of three lower case characters. Using cipher.txt (in this directory), a file containing the encrypted ASCII codes, and the knowledge that the plain text must contain common English words, decrypt the message and find the sum of the ASCII values in the original text.

The following cell shows examples of how to perform XOR in Python and how to go back and forth between characters and integers:

In [1]:
assert 65 ^ 42 == 107
assert 107 ^ 42 == 65
assert ord('a') == 97
assert chr(97) == 'a'

Certain functions in the `itertools` module may be useful for computing permutations:

In [2]:
from itertools import cycle, islice, count, product, takewhile
from string import ascii_lowercase
from collections import Counter

In [3]:
KEY_VALUES = [ord(letter) for letter in ascii_lowercase]
KEYS = [list(key) for key in product(KEY_VALUES, repeat=3)]

In [4]:
COMMON_WORDS = {'I', 'a', 'able', 'about', 'above', 'after', 'all', 'also', 'an', 'and', 'any', 'as', 'ask', 'at', 'back', 'bad', 'be', 'because', 'beneath', 'big', 'but', 'by', 'call', 'can', 'case', 'child', 'come', 'company', 'could', 'day', 'different', 'do', 'early', 'even', 'eye', 'fact', 'feel', 'few', 'find', 'first', 'for', 'from', 'get', 'give', 'go', 'good', 'government', 'great', 'group', 'hand', 'have', 'he', 'her', 'high', 'him', 'his', 'how', 'if', 'important', 'in', 'into', 'it', 'its', 'just', 'know', 'large', 'last', 'leave', 'life', 'like', 'little', 'long', 'look', 'make', 'man', 'me', 'most', 'my', 'new', 'next', 'no', 'not', 'now', 'number', 'of', 'old', 'on', 'one', 'only', 'or', 'other', 'our', 'out', 'over', 'own', 'part', 'people', 'person', 'place', 'point', 'problem', 'public', 'right', 'same', 'say', 'see', 'seem', 'she', 'small', 'so', 'some', 'take', 'tell', 'than', 'that', 'the', 'their', 'them', 'then', 'there', 'these', 'they', 'thing', 'think', 'this', 'time', 'to', 'try', 'two', 'under', 'up', 'us', 'use', 'want', 'way', 'we', 'week', 'well', 'what', 'when', 'which', 'who', 'will', 'with', 'woman', 'work', 'world', 'would', 'year', 'you', 'young', 'your'}

In [5]:
def take(n, iterable):
    yield from islice(iterable, n)

def extend(n, iterable):
    yield from take(n, cycle(iterable))
    
def chunk_by(pred, codes):
    codes_iter = iter(codes)
    while True:
        result = list(takewhile(pred, codes_iter))
        if len(result) == 0:
            break
        yield result

def char_codes_to_str(char_codes):
    result = ""
    for code in char_codes:
        result += chr(code)
    return result

def decode_line(line, key):
    transformed = transform(get_codes(line), key)
    return char_codes_to_str(transformed)

def str_to_char_codes(input_str):
    for char in input_str:
        yield ord(char)

def get_codes(line_str):
    for item in line_str.split(','):
        yield int(item.strip())

def transform(codes, key):
    codes = list(codes)
    num_codes = len(codes)
    pairs = zip(codes, extend(num_codes, key))
    yield from (pair[0] ^ pair[1] for pair in pairs)

def count_common_words(sentence_codes_iter):
    for sentence_codes in sentence_codes_iter:
        sentence = char_codes_to_str(sentence_codes)
        common_test = map(lambda word: word.lower() in COMMON_WORDS, sentence.split())
        yield sum(common_test)

In [6]:
%%time

key_results = Counter()
CAREFUL_SEARCH = True

with open(file='data/cipher.txt', mode='r') as cipher_file:
    for encrypted_line in cipher_file:
        encrypted_values = list(get_codes(encrypted_line))
        for key in KEYS:
            transformed_values = transform(encrypted_values, key)
            sentences_chunks = chunk_by(lambda x: chr(x) != '.', transformed_values)
            counts = count_common_words(sentences_chunks)
            if not CAREFUL_SEARCH:
                counts = takewhile(lambda count: count > 0, counts)
            for count in counts:
                key_results[tuple(key)] += count

CPU times: user 48.1 s, sys: 15.6 ms, total: 48.1 s
Wall time: 48.1 s


In [7]:
ordered_results = sorted(list(key_results.items()), key=lambda x: x[1], reverse=True)
top = take(2, ordered_results)

with open(file='data/cipher.txt', mode='r') as cipher_file:
    for line in cipher_file:
        for top_result in top:
            decode = decode_line(line, top_result[0])
            print("Key:", top_result[0], "->", tuple(list(map(lambda x: chr(x), top_result[0]))))
            print("# Common:", top_result[1])
            print("Text:", decode)
            print("ASCII Sum:", sum(str_to_char_codes(decode)))
            print()

Key: (103, 111, 100) -> ('g', 'o', 'd')
# Common: 111
Text: (The Gospel of John, chapter 1) 1 In the beginning the Word already existed. He was with God, and he was God. 2 He was in the beginning with God. 3 He created everything there is. Nothing exists that he didn't make. 4 Life itself was in him, and this life gives light to everyone. 5 The light shines through the darkness, and the darkness can never extinguish it. 6 God sent John the Baptist 7 to tell everyone about the light so that everyone might believe because of his testimony. 8 John himself was not the light; he was only a witness to the light. 9 The one who is the true light, who gives light to everyone, was going to come into the world. 10 But although the world was made through him, the world didn't recognize him when he came. 11 Even in his own land and among his own people, he was not accepted. 12 But to all who believed him and accepted him, he gave the right to become children of God. 13 They are reborn! This is not 